In [1]:
from pymongo import MongoClient
import json
import pandas as pd
import numpy as np


client = MongoClient('mongodb://localhost:27017/')
db = client.companies


In [2]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies')

In [3]:
filtered_companies = db.filtered_companies.find()

In [4]:
filtered_companies = pd.DataFrame(list(filtered_companies))


In [5]:
filtered_companies.head()

,_id,category_code,deadpooled_year,description,founded_year,funding_rounds,ipo,lat,lng,name,number_of_employees,offices,oficina_principal,tag_list,totalOffices,total_money_raised
0,5d2701c60f9fc4a7eb3c362e,software,None,Private Academic Networks,2008,"[{'funded_year': 2005, 'id': 202, 'raised_amou...",None,37.782263,-122.392142,GoingOn,40.0,"[{'description': 'GoingOn Networks, Inc.', 'lo...","{'coordinates': [-122.392142, 37.782263], 'typ...","socialnetworking, enterprise, social-media, so...",1,$8.5M
1,5d2701c60f9fc4a7eb3c362f,games_video,None,,2009,[],None,53.707739,10.023246,alluc,7.0,"[{'description': '', 'longitude': 10.023246, '...","{'coordinates': [10.023246, 53.707739], 'type'...","video, piracy, videodatabase",1,$0
2,5d2701c60f9fc4a7eb3c3630,games_video,None,,2008,"[{'funded_year': 2007, 'id': 503, 'raised_amou...",None,47.597965,-122.151158,YouBeQB,8.0,"[{'description': None, 'longitude': -122.15115...","{'coordinates': [-122.151158, 47.597965], 'typ...","youbeqb, fantasyfootball, fantasyleague, sports",1,$0
3,5d2701c60f9fc4a7eb3c3631,web,None,,2008,"[{'funded_year': 2005, 'id': 1270, 'raised_amo...",None,40.740231,-73.985433,BeliefNet,NaN,"[{'description': None, 'longitude': -73.985433...","{'coordinates': [-73.985433, 40.740231], 'type...","beliefs, faith, spirituality",1,$7M
4,5d2701c60f9fc4a7eb3c3632,enterprise,None,NoSQL database managed service,2008,"[{'funded_year': 2010, 'id': 375, 'raised_amou...",None,42.358920,-71.057810,Cloudant,70.0,"[{'description': '', 'longitude': -71.05781, '...","{'coordinates': [-71.05781, 42.35892], 'type':...","cloud-computing, big-data, databases, software...",1,$18.3M


In [6]:
# Aqui miro las categorías de las empresas que hay en mi dataset
ca = filtered_companies['category_code'].unique()
ca

array(['software', 'games_video', 'web', 'enterprise', 'network_hosting',
       'other', 'ecommerce', 'hospitality', 'search', 'mobile',
       'analytics', 'hardware', 'photo_video', 'advertising', 'design',
       'education', 'public_relations', 'social', 'sports', 'consulting',
       'messaging', 'cleantech', 'biotech', 'news', 'travel', 'security',
       'semiconductor', 'music', None, 'real_estate', 'finance', 'health',
       'fashion', 'legal', 'medical', 'nanotech'], dtype=object)

In [7]:
# Resultado : {None,'advertising','analytics','biotech','cleantech','consulting','design','ecommerce','education',
# 'enterprise','fashion','finance','games_video','hardware','health','hospitality','legal','medical','messaging',
# 'mobile','music','nanotech','network_hosting','news','other','photo_video','public_relations','real_estate',
# 'search','security','semiconductor','social','software','sports','travel','web'}
discard_cat= [None,'education','fashion','health','hospitality','legal','medical','other',
              'real_estate','travel']

In [8]:
# Como por falta de tiempo no voy a usar los dataframes en los que figuran las empresas que no quiero, he 
# visto que necesito reducir las empresas, ya que no voy a aplicar los criterios de descarte.

filtered_companies2 = db.filtered_companies.find({'$and':[
                    {'category_code':{'$not':{'$eq':None}}}, {'category_code':{'$not':{'$eq':'education'}}},
                    {'category_code':{'$not':{'$eq':'fashion'}}}, {'category_code':{'$not':{'$eq':'health'}}},
                    {'category_code':{'$not':{'$eq':'hospitality'}}}, {'category_code':{'$not':{'$eq':'legal'}}},
                    {'category_code':{'$not':{'$eq':'medical'}}}, {'category_code':{'$not':{'$eq':'other'}}},
                    {'category_code':{'$not':{'$eq':'real_estate'}}}, {'category_code':{'$not':{'$eq':'travel'}}}
                    ]})

In [9]:
filtered_companies2 = pd.DataFrame(list(filtered_companies2))

In [10]:
filtered_companies2.head(2)


,_id,category_code,deadpooled_year,description,founded_year,funding_rounds,ipo,lat,lng,name,number_of_employees,offices,oficina_principal,tag_list,totalOffices,total_money_raised
0,5d2701c60f9fc4a7eb3c362e,software,None,Private Academic Networks,2008,"[{'funded_year': 2005, 'id': 202, 'raised_amou...",None,37.782263,-122.392142,GoingOn,40.0,"[{'description': 'GoingOn Networks, Inc.', 'lo...","{'coordinates': [-122.392142, 37.782263], 'typ...","socialnetworking, enterprise, social-media, so...",1,$8.5M
1,5d2701c60f9fc4a7eb3c362f,games_video,None,,2009,[],None,53.707739,10.023246,alluc,7.0,"[{'description': '', 'longitude': 10.023246, '...","{'coordinates': [10.023246, 53.707739], 'type'...","video, piracy, videodatabase",1,$0


In [11]:
filtered_companies2.to_json('./filtered_companies2.json', orient='records', default_handler=str)

In [12]:
def findNear(geopoint, radio_max_meters=2000):
    return db.filtered_companies2.find({"oficina_principal":{"$near":{
           "$geometry": geopoint,
           "$maxDistance": radio_max_meters,
         }
       }
    })


radio_max_meters=2000
num_offices = pd.DataFrame()

In [13]:
punto = filtered_companies2['oficina_principal'][8]
punto

{'coordinates': [-118.461884, 34.031764], 'type': 'Point'}

In [14]:
lst = list(findNear(punto, radio_max_meters))
lst

[{'_id': '5d2701c60f9fc4a7eb3c3637',
  'category_code': 'games_video',
  'deadpooled_year': None,
  'description': 'How-To Video Search',
  'founded_year': 2008,
  'funding_rounds': [{'funded_day': 31,
    'funded_month': 1,
    'funded_year': 2008,
    'id': 3375,
    'investments': [{'company': None,
      'financial_org': {'name': 'General Catalyst Partners',
       'permalink': 'general-catalyst-partners'},
      'person': None}],
    'raised_amount': None,
    'raised_currency_code': None,
    'round_code': 'a',
    'source_description': 'Mercurial TV Exec Launches WonderHowTo With VC Backing',
    'source_url': 'http://www.thealarmclock.com/mt/archives/2008/01/mercurial_tv_ex.html'}],
  'ipo': None,
  'lat': 34.031764,
  'lng': -118.461884,
  'name': 'WonderHowTo',
  'number_of_employees': 12.0,
  'offices': [{'address1': '1832 Franklin Street',
    'address2': '',
    'city': 'Santa Monica',
    'country_code': 'USA',
    'description': None,
    'latitude': 34.031764,
    'long

In [22]:
#Aqui hacer un ciclo for con la función dentro para que extraiga lo que quiero

lst_of_num_companies =[]
for comp in filtered_companies2['oficina_principal']:
    lst_of_num_companies.append(len(list(findNear(comp, radio_max_meters))))
print(type(lst_of_lists))

filtered_companies2['offices_near']=lst_of_num_companies

<class 'list'>


In [27]:
definitive_companies = filtered_companies2.sort_values('offices_near', ascending=False)

In [28]:
definitive_companies.head()


,_id,category_code,deadpooled_year,description,founded_year,funding_rounds,ipo,lat,lng,name,number_of_employees,offices,oficina_principal,tag_list,totalOffices,total_money_raised,offices_near
411,5d2701c60f9fc4a7eb3c37dc,ecommerce,None,Spa and Well-being social network,2008,"[{'funded_year': 2008, 'id': 3397, 'raised_amo...",None,37.786910,-122.404403,InnerRewards,9.0,"[{'description': 'West Coast', 'longitude': -1...","{'coordinates': [-122.404403, 37.78691], 'type...","wellness, women, social-networking, spas, ecom...",2,$1M,77
493,5d2701c60f9fc4a7eb3c3839,ecommerce,None,Spa and Well-being social network,2008,"[{'funded_year': 2008, 'id': 3397, 'raised_amo...",None,37.786910,-122.404403,InnerRewards,9.0,"[{'description': 'West Coast', 'longitude': -1...","{'coordinates': [-122.404403, 37.78691], 'type...","wellness, women, social-networking, spas, ecom...",2,$1M,77
1277,5d2701c60f9fc4a7eb3c3b9c,biotech,None,Cancer management,2008,"[{'funded_year': 2008, 'id': 6363, 'raised_amo...",None,37.787131,-122.404107,ACT Biotech,NaN,"[{'description': '', 'longitude': -122.4041075...","{'coordinates': [-122.4041075, 37.7871306], 't...","biotech, development, cancer, drugs, medical, ...",1,$27M,77
871,5d2701c60f9fc4a7eb3c39d4,web,None,Social Recruiting and Job Search Site,2008,"[{'funded_year': 2009, 'id': 4902, 'raised_amo...",None,37.788796,-122.409710,KODA,15.0,"[{'description': 'San Francisco', 'longitude':...","{'coordinates': [-122.40971, 37.788796], 'type...",,2,$4.5M,76
38,5d2701c60f9fc4a7eb3c3657,web,None,None,2008,[],None,37.780951,-122.402502,Heavy Electrons,NaN,"[{'description': '', 'longitude': -122.4025016...","{'coordinates': [-122.4025016, 37.7809512], 't...",,1,$0,71


In [29]:
definitive_companies.to_json('./definitive_companies.json', orient='records', default_handler=str)